# Inject contraindications JSON into SULO exercise ontology
This notebook loads *exercise-domain* Turtle file, reads a JSON table of
health‑condition contraindications, injects the missing SULO‑compliant triples,
and saves an updated TTL.

**Requirements**: `rdflib>=6.3` (install in the first cell).

In [6]:

%pip install rdflib>=6.3

zsh:1: 6.3 not found
Note: you may need to restart the kernel to use updated packages.


In [7]:
from pathlib import Path
import json
import re
import argparse
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, XSD, OWL

EX = Namespace("http://example.org/exercise#")
SULO = Namespace("https://w3id.org/sulo/")

ATTR_CLASSES = {
    "ImpactLevel": EX.ImpactLevelQuality,
    "Intensity": EX.IntensityQuality,
    "BalanceRequirement": EX.BalanceRequirementQuality,
    "JointInvolvement": EX.JointInvolvementQuality,
}


def slug(text: str) -> str:
    """Safe URI suffix."""
    return re.sub(r'[^A-Za-z0-9]+', '_', text.strip()).strip('_')


def ensure(g: Graph, uri: URIRef, rdf_type: URIRef, label: str | None = None):
    """Ensure a URI exists in the graph with given type and label."""
    if (uri, RDF.type, None) not in g:
        g.add((uri, RDF.type, rdf_type))
        if label:
            g.add((uri, RDFS.label, Literal(label, lang='en')))


In [8]:
def inject_prescriptions(g, rows):
    """
    Add variation + prescription triples to *existing* graph `g`,
    making variations subclasses of health conditions and all other
    attributes represented by avoid relationships.
    `rows` is a list of dicts loaded from one or many JSON files.
    """
    ensure(g, EX.HealthConditionQuality, OWL.Class)
    g.add((EX.HealthConditionQuality, RDFS.subClassOf, SULO.Quality))

    condition_variations = {}
    for row in rows:
        cond_label = row["HealthCondition"].strip()
        if cond_label not in condition_variations:
            condition_variations[cond_label] = []
        condition_variations[cond_label].append(row)
    
    for cond_label, variations in condition_variations.items():

        cond_class = EX[f"{slug(cond_label)}Quality"]
        ensure(g, cond_class, OWL.Class, f"{cond_label} condition quality")
        g.add((cond_class, RDFS.subClassOf, EX.HealthConditionQuality))
        
        for row in variations:
            var_label = row["Variation"]
            
            var_class = EX[f"{slug(cond_label)}_{slug(var_label)}Quality"]
            ensure(g, var_class, OWL.Class, f"{cond_label} - {var_label}")
            g.add((var_class, RDFS.subClassOf, cond_class))
            
            avoid_set_uri = EX[f"AvoidSet_{slug(cond_label)}_{slug(var_label)}"]
            ensure(g, avoid_set_uri, SULO.Set, f"avoid attributes for {cond_label} - {var_label}")
            
            g.add((var_class, SULO.refersTo, avoid_set_uri))
            
            var_info_uri = EX[f"Variation_{slug(cond_label)}_{slug(var_label)}"]
            ensure(g, var_info_uri, SULO.InformationObject, f"{var_label} variation of {cond_label}")
            g.add((var_info_uri, SULO.hasValue, Literal(var_label, datatype=XSD.string)))
            g.add((var_class, SULO.hasValue, Literal(var_label, datatype=XSD.string)))
            
            for key, q_class in ATTR_CLASSES.items():

                if key == "JointInvolvement":
                    continue
                    
                val = row.get(key, [])
                
                if isinstance(val, str):
                    if val.upper() != "NULL":
                        val = [val]
                    else:
                        val = []
                
                for value in val:
                    indiv_uri = EX[f"{slug(value)}{key}"]
                    ensure(g, indiv_uri, SULO.InformationObject,
                           f"{value.lower()} {key.lower()}")
                    g.add((indiv_uri, SULO.hasValue,
                           Literal(value.lower(), datatype=XSD.string)))
                    g.add((indiv_uri, SULO.refersTo, q_class))
                    g.add((avoid_set_uri, SULO.refersTo, indiv_uri))
            
            joint_vals = row.get("JointInvolvement", [])
            if isinstance(joint_vals, str) and joint_vals.upper() != "NULL":
                joint_vals = [joint_vals]
                
            if joint_vals and joint_vals[0].lower() != "none":
                for joint in joint_vals:
                    q_uri = EX[f"{slug(joint)}InvQ"]
                    v_uri = EX[f"{slug(joint)}InvV"]
                    ensure(g, q_uri, EX.JointInvolvementQuality)
                    ensure(g, v_uri, SULO.InformationObject, f"{joint} involvement")
                    g.add((v_uri, SULO.hasValue, Literal(joint, datatype=XSD.string)))
                    g.add((v_uri, SULO.refersTo, q_uri))
                    g.add((avoid_set_uri, SULO.refersTo, v_uri))

In [9]:
def inject_prescriptions_multi(schema_ttl: Path, out_ttl: Path, *json_files: Path):
    """
    Merge prescription rows from any number of JSON files
    into the exercise-domain ontology.
    """
    g = Graph()
    g.parse(schema_ttl, format="turtle")

    all_rows = []
    for jfile in json_files:
        with open(jfile, encoding="utf-8") as fh:
            all_rows.extend(json.load(fh))

    inject_prescriptions(g, all_rows) 
    g.serialize(out_ttl, format="turtle")
    print(f"✓ Saved merged ontology to {out_ttl}")


In [10]:

schema = Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/sulo_1.ttl')
out    = Path('sulo_2.ttl')
json1  = Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/HealthCondition_Contraindications/amputees_condition.json')
json2  = Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/HealthCondition_Contraindications/arthritis_condition.json')
json3  = Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/HealthCondition_Contraindications/cerebral_palsy_condition.json')
json4  = Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/HealthCondition_Contraindications/chronic_back_pain_condition.json')
json5  = Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/HealthCondition_Contraindications/chronic_fatigue_syndrome_condition.json')
json6  = Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/HealthCondition_Contraindications/diabetes.json')
json7  = Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/HealthCondition_Contraindications/fibromyalgia_condition.json')
json8  = Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/HealthCondition_Contraindications/frailty_condition.json')
json9  = Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/HealthCondition_Contraindications/multiple_sclerosis.json')
json10  = Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/HealthCondition_Contraindications/muscular_dystrophy_condition.json')
json11  = Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/HealthCondition_Contraindications/myasthenia_gravis.json')
json12  = Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/HealthCondition_Contraindications/myasthenia_gravis.json')
json13  = Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/HealthCondition_Contraindications/myopathy_condition.json')
json14  = Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/HealthCondition_Contraindications/obesity.json')
json15  = Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/HealthCondition_Contraindications/osteoporosis_condition.json')
json16  = Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/HealthCondition_Contraindications/paraplegia_wheelchair-bound_condition.json')
json17  = Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/HealthCondition_Contraindications/parkinson_disease_condition.json')
json18  = Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/HealthCondition_Contraindications/peripheral_neuropathy_condition.json')
json19  = Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/HealthCondition_Contraindications/spinal_cord_injury.json')
json20  = Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/HealthCondition_Contraindications/stroke_condition.json')
json21  = Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/HealthCondition_Contraindications/traumatic_brain_injury.json')
json22  = Path('/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/HealthCondition_Contraindications/wheelchair_use.json')




inject_prescriptions_multi(schema, out, json1, json2, json3, json4, json5, json6, json7, json8, json9,
                          json10, json11, json12, json13, json14, json15, json16, json17, json18,
                          json19, json20, json21, json22)



FileNotFoundError: [Errno 2] No such file or directory: '/Users/alexandruvalah/Desktop/Thesis/ContraindicationsGeneration/attribute_based/sulo_1.ttl'